In [1]:
from ssri_interactions.interactions.loaders import SpontaneousActivityLoader, StateInteractionsLoader
from ssri_interactions.interactions.preprocessors import InteractionsPreprocessor
from ssri_interactions.interactions.pairwise import PairwiseCorr
from ssri_interactions.interactions.graph_clustering import df_to_graph
from ssri_interactions.config import Config, ExperimentInfo
import numpy as np
from ssri_interactions.interactions.graph import GraphAttributes, NodeAttributes
import pandas as pd
import seaborn as sns
from ssri_interactions.io import load_derived_generic, load_distances
from ssri_interactions.transforms.graph import GraphTransformer
import matplotlib.pyplot as plt
from ssri_interactions.plots import PAL_GREY_BLACK
from scipy.stats import chi2_contingency

fig_dir = Config.fig_dir
sns.set_theme(style="ticks", context="paper")

c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### Get Graph Results

- Graph DF (SWP, Avg Clutering etc)
- Node (Centrality, NT, Clustering etc)
- Edge DF (Weight, Distance, NT Combo, Same NT)

In [5]:
def load_responders():
    slow_responders_shock = (
        load_derived_generic("slow_ts_foot_shock_unit_responders_pre_to_shock.csv")
        [["neuron_id", "response"]]
        .rename(columns={"response": "response_fs_slow"})
    )
    fast_responders_first_window = (
        load_derived_generic("fast_fs_foot_shock_unit_responders.csv")
        [["neuron_id", "fs_fast_response"]]
    )
    fast_responders_second_window = (
        load_derived_generic("fast_fs_foot_shock_unit_responders_second_window.csv")
        [["neuron_id", "response_second_window"]]
    )
    neuron_types = load_derived_generic("neuron_types.csv")[["neuron_id", "neuron_type", "session_name", "group"]]
    df_responders = (
        slow_responders_shock
        .merge(fast_responders_first_window, how="outer")
        .merge(fast_responders_second_window, how="outer")
        .merge(neuron_types, how="left")
    )
    return df_responders


def get_graph_stats(session_name, bin_width, graph_transformer, shuffle):
        graph_attrs = GraphAttributes(distance_from_edge="bounded")
        node_attrs = NodeAttributes()
        loader = SpontaneousActivityLoader(
            session_name=session_name, bin_width=bin_width, block="pre", t_start=0, t_stop=1800, 
            shuffle=shuffle
        )
        preprocessor = InteractionsPreprocessor(z=False)
        pairwise = PairwiseCorr(rectify=True)
        spikes = preprocessor(loader())
        df_affinity = (
            pairwise.fit(spikes)
            .get_adjacency_df()
        )
        G = df_to_graph(df_affinity, rename_nodes=True)
        graph_stats = graph_attrs.get_graph_attributes(G).assign(
            session_name=session_name,  bin_width=bin_width, shuffle=shuffle)
        node_stats = node_attrs.get_node_attributes(G, node_name="neuron_id").assign(
            session_name=session_name,  bin_width=bin_width, shuffle=shuffle
        )
        edge_stats = (
            graph_transformer
            .graph_to_edge_df(G)
            .assign(session_name=session_name, bin_width=bin_width, shuffle=shuffle)
        )
        return graph_stats, node_stats, edge_stats

In [6]:
neuron_types = load_derived_generic("neuron_types.csv")
df_ensembles = (
    load_derived_generic("ensembles/spont - ensembles - true.csv")
    .assign(in_ensemble = lambda x: np.where(x["ensemble_id"] == -1, False, True))
)
df_responders = load_responders()
df_distance = load_distances()
sessions = neuron_types["session_name"].dropna().unique()

g_transform = GraphTransformer(
    relabel_nodes=True,
    weight_attr="weight",
    neuron_types=neuron_types,
    df_distance=df_distance,
    df_ensemble=df_ensembles,
)

graph_dfs = []
node_dfs = []
edge_dfs = []

for shuffle in [True, False]:
    for bin_width in [0.05, 0.1, 0.5, 1]:
        for session in sessions:
            graph_stats, node_stats, edge_stats = get_graph_stats(session, bin_width, g_transform, shuffle=shuffle)
            graph_dfs.append(graph_stats)
            node_dfs.append(node_stats)
            edge_dfs.append(edge_stats)

df_graph = (
    pd.concat(graph_dfs)
    .reset_index(drop=True)
    .merge(neuron_types[["session_name", "group"]].drop_duplicates(),)
)

df_node = (
    pd.concat(node_dfs)
    .reset_index(drop=True)
    .merge(neuron_types[["neuron_id", "neuron_type", "group"]])
    .merge(df_ensembles[["neuron_id", "in_ensemble"]])
)
df_edge = (
    pd.concat(edge_dfs)
    .reset_index(drop=True)
    .merge(neuron_types[["session_name", "group"]].drop_duplicates(),)
)

In [7]:
dd = Config.derived_data_dir / "graph"
dd.mkdir(parents=True, exist_ok=True)

df_graph.to_parquet(dd / "spont - graph.parquet", index=False)
df_node.to_parquet(dd / "spont - node.parquet", index=False)
df_edge.to_parquet(dd / "spont - edge.parquet", index=False)
df_responders.to_parquet(dd / "spont - responders.parquet", index=False)